<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark10_movement_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')


Mounted at /content.gdrive


In [20]:
import json
import numpy as np
import cv2

with open('/content.gdrive/MyDrive/Test2/exp53/predictions.json', "r") as fp:
    data = json.load(fp)

print(type(data["x_center"]))
print(data["x_center"])
# convert lists to numpy arrays
x_center = np.array(data["x_center"])
y_center = np.array(data["y_center"])
x1 = np.array(data["x1"])
x2 = np.array(data["x2"])
y1 = np.array(data["y1"])
y2 = np.array(data["y2"])
det = np.array(data["det"])
width = 1920
hight = 1080


<class 'list'>
[1276.4999771118164, 1277.9999542236328, 347.49999046325684, 1145.0000381469727, 416.49999618530273, 956.0000038146973, 720.9999847412109, 1232.4999618530273, 346.5000057220459, 416.49999618530273, 1276.9999694824219, 1144.5000457763672, 956.9999885559082, 720.4999923706055, 1456.4999771118164, 416.49999618530273, 1276.4999771118164, 1145.5000305175781, 956.0000038146973, 347.49999046325684, 720.4999923706055, 1155.4999923706055, 1275.999984741211, 956.4999961853027, 416.00000381469727, 1145.5000305175781, 348.0000114440918, 720.0, 350.50000190734863, 416.49999618530273, 1275.4999923706055, 956.0000038146973, 1145.0000381469727, 1115.0000381469727, 720.4999923706055, 1095.999984741211, 1276.9999694824219, 350.50000190734863, 416.9999885559082, 956.4999961853027, 1145.0000381469727, 719.5000076293945, 1064.000015258789, 956.9999885559082, 1276.4999771118164, 350.50000190734863, 1145.5000305175781, 416.49999618530273, 720.0, 837.4999809265137, 1277.4999618530273, 956.99998

In [42]:
print(sum(det[:1]))
print(det)
print(sum(det[:0+1]))
print(sum(det[:0])+0)
print(math.sqrt((x_center[sum(det[:0+1])]-x_center[sum(det[:0])+0])**2 + (y_center[sum(det[:0+1])]-y_center[sum(det[:0])+0])**2))

list_soll = [1,0,'X',0,5,0,0,7,6,2,4,6,2,2,5,0,1,4,1,0,2]


7
[7 7 7 7 7 7 7 8 7 7 8 8 8 6 9 6 8 8 8 7 9]
7
0
115.69361771412606


In [57]:
#detect movement
import math
from collections import OrderedDict

athlet_det_index = []
y_centers = []

for y in range(21):
    distance = []
    distance_index = []
    valid_dist_index = []
    y_center_per_det = []
    if y < 20:
        for x in range(det[y+1]):
            y_center_per_det.append(y_center[sum(det[:y])+x])
            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        
              
    else:
        for x in range(det[y-1]):

            for i in range(det[y]):
                distance.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index.append(i)
        #print(distance)
        #print(distance_index)

    print(y_center_per_det)
    y_centers.append(y_center_per_det)
    for i, x in enumerate(distance):
        if x > 15 and x < 50:
            valid_dist_index.append(distance_index[i])
            
    if len(valid_dist_index) > 1:
        valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index)) 
    #print(len(valid_dist_index))
    if len(valid_dist_index) == 1:
        athlet_det_index.append(valid_dist_index[0])
    if len(valid_dist_index) == 0:
        athlet_det_index.append('X')
    if len(valid_dist_index) > 1:
        dist_to_previous_y_center = []
        for i, x in enumerate(valid_dist_index):
            dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_centers[y-1][athlet_det_index[y-1]]))

        closest = min(dist_to_previous_y_center)
        min_index = dist_to_previous_y_center.index(closest)
        print("BP",valid_dist_index[min_index])

        #valid_y = [y_center_per_det[valid_dist_index[0]]]
        #for i, x in enumerate(y_center_per_det):

        #y_centers[athlet_det_index[y-1]]



    





print(valid_dist_index)
print(list_soll)
print(athlet_det_index)
print(y_centers)
print(y_centers[1][2])






[540.9999704360962, 646.0000205039978, 1015.4999756813049, 553.4999871253967, 602.5000190734863, 565.5000185966492, 576.5000152587891]
[648.0000257492065, 1014.9999904632568, 601.9999694824219, 542.0000052452087, 553.9999723434448, 564.9999690055847, 577.4999856948853]
[547.4999713897705, 602.5000190734863, 543.5000252723694, 553.4999871253967, 564.9999690055847, 1012.5, 577.0000004768372]
[663.500018119812, 542.9999756813049, 565.5000185966492, 602.5000190734863, 553.4999871253967, 1012.9999852180481, 577.9999709129333]
[1012.9999852180481, 603.0000042915344, 543.5000252723694, 565.5000185966492, 553.4999871253967, 659.0000224113464, 576.000030040741]
[671.4999747276306, 542.9999756813049, 1012.0000147819519, 602.5000190734863, 565.5000185966492, 553.4999871253967, 577.9999709129333]
[677.0000052452087, 565.5000185966492, 543.5000252723694, 1012.0000147819519, 553.9999723434448, 602.5000190734863, 577.4999856948853, 624.000027179718]
[624.000027179718, 542.9999756813049, 564.999969005